### Building a RAG based Question Answer Tool

- The Task is to build a RAG pipeline to do QnA on a certain set of Documents. Here are the links for the Documents:

    Withdrawal of Legal Tender Character of ₹ 500 and ₹ 1,000: RBI Statement
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=38568
    
    TBills
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=38747
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=38747
    
    minutes of monetary policy 2024
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=57366
          
    monetary penalty
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=57364
    
    VRR
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=57550
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=55274
     
    Minutes of meeting 2023
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=56982
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=56606
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=56252
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=55899
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=55538
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=55267
    
     
    
    Full Auction Result 2016
    
    https://www.rbi.org.in/Scripts/BS_PressReleaseDisplay.aspx?prid=38736



- By looking at the documents, i found that there are multiple monetary policy meetings of 2023 ,2024 and other press release documents. Since there are similar documents for different time periods , it is important to store the time period in the chunks so that one can retrieve the correct chunk for the particular time period.
- In order to store the time period in the chunks, i am extracting the date of the document from the document text and storing it as month-year format since monetary meetings happens once in 3 months and easy to apply filter on month-year rather than on exact date.I am storing this month-year in the metadata along with page number and file name so that i can cite the document name and page number in the output response.
- I am using OpenAI API for using the LLM model and Chroma for vector database.
- I have used Langchain framework for defining different LLM chains and building the RAG pipeline
- I have created a chatbot through which user can easily do QnA about these documents.
- I have downloaded and stored all the documents in the 'data' folder.

#### 1) Importing all the libraries

In [1]:
from pprint import pprint
from datetime import datetime
import os
import getpass
import ast
import uuid

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI

# Prompts
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

# Conversational memory
from langchain.memory import ConversationBufferMemory

# LLM wrapper
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

from langchain.schema.messages import AIMessage, HumanMessage

from langchain.schema import BaseChatMessageHistory
from langchain.schema.messages import (
    BaseMessage,
    messages_from_dict,
    messages_to_dict,
)



#### 2) Adding OpenAI API KEY

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key: ········


#### 3) Defining the LLM models

In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

/opt/anaconda3/envs/genai_dev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
llm_gpt35 = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)

/opt/anaconda3/envs/genai_dev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
llm_gpt4 = ChatOpenAI(model_name='gpt-4-0125-preview', temperature=0)

#### 4) Creating custom LLM chains for different purposes

In [6]:
### This will be used to extract the month & year from the date mentioned in the document as this will be stored in the metadata

date_extraction_prompt_template = """
                     You have to extract the month and year from the date mentioned in the context. Output the result in this format : month-year
                     e.g. : if date mentioned is 22/11/2022 then output should be November-2022
                     if date mentioned is 15th May 2020, then output should be May-2020.
                     Output None if you cannot find month or year.
                     ```\n
                     {context}
                     \n```
                     """

date_extraction_prompt = PromptTemplate(template=date_extraction_prompt_template,input_variables = ["context"])

In [7]:
### We are using StuffDocuments Chain to extract the date from the input document

llm_chain = LLMChain(llm=llm_gpt35, prompt=date_extraction_prompt)
date_extraction_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="context"
)


#### 5) Loading and processing the data

In [8]:
pdf_folder_path = './data'
loaders = []
for file in os.listdir(pdf_folder_path):
    if '.pdf' in file:
        loaders.append(PyPDFLoader(os.path.join(pdf_folder_path, file)))

In [9]:
os.listdir(pdf_folder_path)

['PR1769MPCF562EB05339B404F8DC23D07725EF354.pdf',
 'PR1531MPC028F84F6CF55474FAD12AF5AB0271F8C.pdf',
 'PR210094BA4333A49943578DC244BF2D975D76.pdf',
 'PR88MINUTESOFMPC507ABE3E36B346B6A843FCDCCD9F6207.pdf',
 'PR136980056AFD85C8475286C0BD6B6F02F48C (1).pdf',
 'PR449MINUTESMPC0878FC98204D417C85943F174D1A1A7A.pdf',
 'PR1190C026F89B382C47ABBC3D2E4B1A6E0DA1.pdf',
 'PR136980056AFD85C8475286C0BD6B6F02F48C.pdf',
 'PR1155MINUTESMPCA747B17FC38A4EE593AC2CAF6A97C0E2.pdf',
 'PR802MINUTESMPC82557115808D41C489D15857CA3D1E67.pdf',
 'PR1916MINUTESMPCEC644F0858CB44DA8948B3C027C81C5D.pdf',
 'PR1914ADINATH25D3CEFC29704A9289D6D7BDA04708E5.pdf',
 '.ipynb_checkpoints',
 'PR1358EB308DAB38B24F4EB2725FD44EA7237B.pdf',
 'PR1776533F25F2E77D4C91A1F758C7897D2E00.pdf']

In [10]:
### Extracting month-year from each document and storing it in metadata along with correct page number and file name

all_documents = []
for file_name in tqdm(os.listdir(pdf_folder_path)):
    try:
        loader = PyPDFLoader(os.path.join(pdf_folder_path, file_name))
        docs = loader.load()
        month_year = date_extraction_chain.run(docs[0:1])
        print('Retrieved Month & year from the document : {} are {}'.format(docs[0].metadata['source'],month_year))
        for doc in docs:
            doc.metadata['month_year'] = month_year
            doc.metadata['page'] += 1
            doc.metadata['file_name'] = str(file_name)
        all_documents.extend(docs)
    except:
        pass

  0%|                                                    | 0/15 [00:00<?, ?it/s]/opt/anaconda3/envs/genai_dev/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
  7%|██▉                                         | 1/15 [00:01<00:20,  1.47s/it]

Retrieved Month & year from the document : ./data/PR1769MPCF562EB05339B404F8DC23D07725EF354.pdf are February-2023


 13%|█████▊                                      | 2/15 [00:02<00:16,  1.26s/it]

Retrieved Month & year from the document : ./data/PR1531MPC028F84F6CF55474FAD12AF5AB0271F8C.pdf are December-2023


 20%|████████▊                                   | 3/15 [00:03<00:12,  1.03s/it]

...

Retrieved Month & year from the document : ./data/PR1914ADINATH25D3CEFC29704A9289D6D7BDA04708E5.pdf are February-2024


 93%|████████████████████████████████████████▏  | 14/15 [00:12<00:00,  1.41it/s]

Retrieved Month & year from the document : ./data/PR1358EB308DAB38B24F4EB2725FD44EA7237B.pdf are November-2016


100%|███████████████████████████████████████████| 15/15 [00:14<00:00,  1.04it/s]

Retrieved Month & year from the document : ./data/PR1776533F25F2E77D4C91A1F758C7897D2E00.pdf are February-2023


In [11]:
len(all_documents)

97

In [12]:
all_documents[0].metadata,all_documents[1].metadata

({'source': './data/PR1769MPCF562EB05339B404F8DC23D07725EF354.pdf',
  'page': 1,
  'month_year': 'February-2023',
  'file_name': 'PR1769MPCF562EB05339B404F8DC23D07725EF354.pdf'},
 {'source': './data/PR1769MPCF562EB05339B404F8DC23D07725EF354.pdf',
  'page': 2,
  'month_year': 'February-2023',
  'file_name': 'PR1769MPCF562EB05339B404F8DC23D07725EF354.pdf'})

##### Splitting the documents in chunks
- I am using RecursiveCharacterTextSplitter to split the documents into chunks of chunk size of 2500 with a chunk overlap of 250.
- The reason to use RecursiveCharacterTextSplitter over other TextSplitter is that this splitter splits the paragraph, sentences and words in a better way and is recommended for generic text. You can find more info about it here : https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter/
- I have kept chunk size of 2500 characters as i am using a larger embedding model and i tried with different sizes to make sure i can get the tables in the documents in one chunk and i find 2500 to be optimal for that

In [13]:
### Define the Text Splitter 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2500,
    chunk_overlap = 250
)


In [14]:
splits = text_splitter.split_documents(all_documents)

len(splits)

178

In [15]:
splits[0].page_content, splits[0].metadata

('February  22, 2023  \n                                                                         \nMinutes of the Monetary Policy Committee Meeting, February 6- 8, 2023 \n[Under Section 45ZL of the Reserve Bank of India Act, 1934]  \n \n     The forty first meeting of the Monetary Policy Committee (MPC), constituted under \nsection 45ZB of the Reserve Bank of India Act, 1934, was held during February 6- 8, \n2023.  \n2. The meeting was attended by all the members –  Dr. Shashanka Bhide, Honorary \nSenior Advisor, National Council of Applied Economic Research, Delhi; Dr. Ashima \nGoyal, Emeritus Professor, Indira Gandhi Institute of Development Research, Mumbai; \nProf. Jayanth R. Varma, Professor, Indian Institute of Management, Ahmedabad;           \nDr. Rajiv Ranjan, Executive Director (the  officer of the Reserve Bank nominated by the \nCentral Board under Section 45ZB(2)(c) of the Reserve Bank of India Act, 1934);            \nDr. Michael Debabrata Patra, Deputy Governor in charge 

In [18]:
### To remove duplicate chunks or documents if any before storing in the vector database

### Create a list of unique ids for each document based on the content
ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, split.page_content)) for split in splits]
unique_ids = list(set(ids))

### Ensure that only docs that correspond to unique ids are kept and that only one of the duplicate ids is kept
seen_ids = set()
unique_splits = [split for split, id in zip(splits, ids) if id not in seen_ids and (seen_ids.add(id) or True)]

In [19]:
len(unique_splits)

177

- I am storing the page number and document name in the end of the page content so that when i perform retrieval i can get the document name and page number information in the retrieved chunks

In [20]:
for split in unique_splits:
    page_num = int(split.metadata["page"])
    doc_name = str(split.metadata["file_name"])
    split.page_content += "\ndocument name is " + str(doc_name)
    split.page_content += "\npage " + str(page_num)

In [21]:
print(unique_splits[0].page_content)
print(unique_splits[0].metadata)

February  22, 2023  
                                                                         
Minutes of the Monetary Policy Committee Meeting, February 6- 8, 2023 
[Under Section 45ZL of the Reserve Bank of India Act, 1934]  
 
     The forty first meeting of the Monetary Policy Committee (MPC), constituted under 
section 45ZB of the Reserve Bank of India Act, 1934, was held during February 6- 8, 
2023.  
2. The meeting was attended by all the members –  Dr. Shashanka Bhide, Honorary 
Senior Advisor, National Council of Applied Economic Research, Delhi; Dr. Ashima 
Goyal, Emeritus Professor, Indira Gandhi Institute of Development Research, Mumbai; 
Prof. Jayanth R. Varma, Professor, Indian Institute of Management, Ahmedabad;           
Dr. Rajiv Ranjan, Executive Director (the  officer of the Reserve Bank nominated by the 
Central Board under Section 45ZB(2)(c) of the Reserve Bank of India Act, 1934);            
Dr. Michael Debabrata Patra, Deputy Governor in charge of monetary poli

#### 6) Storing the Chunks into Vector Database

- I am using Chroma Vector Database to store the chunk embeddings. Chroma is opensource and easy to use and we can also store the vectors locally in memory
- You can read more about Chroma Vector Database here : https://python.langchain.com/docs/integrations/vectorstores/chroma/

In [22]:
persist_directory = 'chroma_db/'

### If creating vector store for the first time use this else load directly from vectorstore stored locally
### Create the vector store

vectordb = Chroma.from_documents(
    documents=unique_splits,
    ids = unique_ids,
    embedding=embeddings,
    persist_directory=persist_directory
)

### if the vectorstore is already created and stored locally, use this command to load it insted of creating the embeddings again

# vectordb =  Chroma(persist_directory=persist_directory, embedding_function=embeddings)


In [23]:
print(vectordb._collection.count())

177


#### 7) Creating Custom Prompts for different tasks

In [24]:
### This is the prompt to extract the month-year from the date or time period mentioned in the user question
### The reason to create this again is that there can be multiple dates/time-periods mentioned by the user in the query whereas in one document we will have only one date

ques_date_extraction_prompt_template = """
                     You have to extract the month and year from the date mentioned in the context. Output the result as a list containing month and year in this format : month-year
                     e.g. : if date mentioned is 22/11/2022 then output should be ['November-2022']
                     if date mentioned is 15th May 2020, then output should be ['May-2020'].
                     if there are multiple dates mentioned then output all of them.
                     Output None if you cannot any date mentioned in the question.
                     ```\n
                     {context}
                     \n```
                     """

ques_date_extraction_prompt = PromptTemplate(template=ques_date_extraction_prompt_template,input_variables = ["context"])

In [25]:
### This is the prompt to rephrase the question using the previous chat history and new question asked by the user
### The purpose to rephrase the question is to get the context of the previous conversation/chat in the current question

contextualize_q_template = """Given the following conversation, chat history and a follow up question, rephrase the follow up question to be a standalone question, in its original language with clear and concise meaning and intent. \n
Maintain the context of the previous conversation in the final standalone question.

Chat history:
{chat_history}

Followup question:
{question}

Standalone question:
"""
contextualize_q_prompt = PromptTemplate(input_variables=['chat_history','question'],template = contextualize_q_template)

In [26]:
### This is the prompt for getting the final output from the retrieved chunks which are passed as context
### This prompt will be used for the final rag chain

qa_prompt_template = """You are an assistant for question-answering tasks. \
We have provided context information below in between triple backticks. \
Given this information and ONLY this information, please answer this question: {question}

```
{context}
```
If you don't know the answer, just say that you don't know, don't try to make up an answer. \
Please also tell me in what pdf is that information, the page and the section so that I can easily find it whenever it applies.
"""

qa_prompt = PromptTemplate.from_template(qa_prompt_template)


In [27]:
prompt_list = [qa_prompt,contextualize_q_prompt,ques_date_extraction_prompt]

#### 8) Creating Chatbot Class by combining all the components

In [40]:
class Chatbot:
    """
    This class represents a Chatbot that can query documents and interact with users.

    Attributes:
    vectordb (object): An instance of the VectorDB database.
    qa_prompt (str): The prompt used for the QA chain.
    contextualize_q_prompt (str): The prompt used for the question rephraser chain.
    ques_date_extractor_prompt (str): The prompt used for the question date extractor chain.
    chat_history (list): A list to store the history of the chat.
    welcome_message (str): The welcome message of the chatbot.

    Methods:
    get_llm(model): Returns a language model based on the model name.
    create_ques_date_extractor_chain(llm): Creates a question date extractor chain.
    create_ques_rephraser_chain(llm): Creates a question rephraser chain.
    query_document(question): Queries a document based on the question asked by the user.
    start_bot(): Starts the chatbot.
    """
    
    def __init__(self,prompt_list,
                vectordb
                ):
        """
        The constructor for the Chatbot class.

        Parameters:
        prompt_list (list): A list of prompts used for different chains.
        vectordb (object): An instance of the VectorDB database.
        """
        
        self.vectordb = vectordb
        self.qa_prompt = prompt_list[0]
        self.contextualize_q_prompt = prompt_list[1]
        self.ques_date_extractor_prompt = prompt_list[2]
        self.chat_history = []
        self.welcome_message = "Hi, How can i help you ?"
    
    def get_llm(self,model='gpt4'):
        """
        This function returns a language model based on the model name.
        
        Parameters:
        model (str): The name of the model. Default is 'gpt4'.
        
        Returns:
        object: An instance of the ChatOpenAI class with the specified model.
        """
        if model == 'gpt35':
            return ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)
        else:
            return ChatOpenAI(model_name='gpt-4-0125-preview', temperature=0)
    
    def create_ques_date_extractor_chain(self, llm):
        return LLMChain(llm=llm, prompt=self.ques_date_extractor_prompt)

    def create_ques_rephraser_chain(self,llm):
        return LLMChain(llm=llm, prompt=self.contextualize_q_prompt)

    def query_document(self,question):
        """
        This function is used to query a document based on the question asked by the user. 
    
        Parameters:
        question (str): The question asked by the user.
    
        Returns:
        dict: The result of the query which includes the answer to the question.
    
        Functionality:
        1. It first gets the language models 'gpt35' and 'gpt4'.
        2. It then creates a question date extractor chain using 'gpt35'.
        3. It extracts the month and year from the question.
        4. Based on the extracted month and year, it defines metadata filters.
        5. It defines a ChromaDB retriever with the defined metadata filters.
        6. It creates a question rephraser chain using 'gpt4'.
        7. It loads a QA chain with sources using 'gpt4'.
        8. It creates a RAG chain using the retriever, question rephraser chain, and the QA chain.
        9. It asks the question and gets the response using the RAG chain.
        10. It adds the question and the response to the chat history.
        11. Finally, it returns the result of the query.
        """
        
        llm_gpt35 = self.get_llm('gpt35')
        llm_gpt4 = self.get_llm('gpt4')

        ### Question Date extractor chain
        ques_date_extractor_chain = self.create_ques_date_extractor_chain(llm_gpt35)

        ### Extracting month-year from the question
        ques_month_year = ques_date_extractor_chain.run(question)

        ### Defining metadata filters based on the extracted month-year
        if ques_month_year=='None':
            metadata_filter = {}
        else:
            ques_month_year = ast.literal_eval(ques_month_year)
            metadata_filter = {"month_year":{"$in":ques_month_year}}
            
        ### Defining ChromaDB retriever
        retriever=self.vectordb.as_retriever(search_kwargs={"filter":metadata_filter,"k":6})     
        
        ### Question rephraser chain
        question_rephraser_chain = self.create_ques_rephraser_chain(llm_gpt4)

        ### QA chain
        doc_chain = load_qa_with_sources_chain(llm_gpt4,prompt=self.qa_prompt,document_variable_name='context')

        ### RAG chain : using the ConversationalRetrievalChain for conversational capability
        rag_chain = ConversationalRetrievalChain(
            retriever=retriever,
            question_generator=question_rephraser_chain,
            combine_docs_chain=doc_chain,
            return_source_documents='True',
            output_key='answer'
        )

        ### Asking the question and getting the response
        result = rag_chain({"question": question, "chat_history": self.chat_history})

        ### Adding the question and the response in the chat history
        self.chat_history.append((question, result["answer"]))
        
        return result
        
    def detect_exit(self, message):
        """
        This function checks if the user's message contains any exit indicators.
    
        Parameters:
        message (str): The message from the user.
    
        Returns:
        bool: True if an exit indicator is found in the message, False otherwise.
    
        Functionality:
        It checks if the user's message contains any of the predefined exit indicators ("thank", "nice"). If an exit indicator is found, it returns True, indicating that the user may want to end the conversation. Otherwise, it returns False.
        """
        
        exit_indicators = ["thank", "nice"]
        for indicator in exit_indicators:
            if (indicator.lower() in message.lower()):
                return True
        return False
        
    def start_bot(self):
        """
        This function is used to start the chatbot. 
    
        Functionality:
        1. It first prints the welcome message and initializes the chat history.
        2. It then enters a loop where it continuously waits for user input.
        3. When a user question is received, it queries the document using the question.
        4. It then prints the bot's response to the question.
        5. This process continues until the program is manually stopped.
    
        Note: 
        This function does not take any parameters and does not return anything. It's an infinite loop that keeps the chatbot running and responding to user queries.
        """
        
        print(self.welcome_message + "\n\n")
        
        self.chat_history = []
        
        while True: 

            user_question = input("*** User Question *** : ")
            
            if self.detect_exit(user_question):
                break
                
            bot_response = self.query_document(user_question)
            
            print("*** Bot Response *** : ", bot_response['answer'], "\n ")
           

#### 9) Let's do some Question Answering

In [48]:
### Initialize the Chatbot

chatbot = Chatbot(prompt_list,vectordb)

##### We can start the chatbot using chatbot.start_bot() and it will start a continuous question answer bot through a continuous running cell, in order to stop it you need to type Thank you or nice

In [49]:
chatbot.start_bot()

Hi, How can i help you ?




*** User Question *** :  What are the type of documents in the database ?


*** Bot Response *** :  The types of documents in the database are:

1. Press Releases from the Reserve Bank of India, detailing various financial and economic measures, updates, and results of auctions. Examples include announcements regarding the withdrawal of legal tender status for certain banknotes, results of State Development Loans auctions, and Treasury Bills auction results.

2. Minutes of the Monetary Policy Committee meetings, providing insights into economic assessments, inflation projections, growth forecasts, and policy decisions.

For instance:
- The document named "PR1190C026F89B382C47ABBC3D2E4B1A6E0DA1.pdf" is a press release about the withdrawal of legal tender character of `500 and `1,000 banknotes. (Source: ./data/PR1190C026F89B382C47ABBC3D2E4B1A6E0DA1.pdf, page 1)
  
- The document named "PR1916MINUTESMPCEC644F0858CB44DA8948B3C027C81C5D.pdf" contains minutes from a Monetary Policy Committee meeting discussing economic conditions and policy decisions. (Source: ./dat

*** User Question *** :  On which bank did RBI implied monetary penalty?


*** Bot Response *** :  The Reserve Bank of India imposed a monetary penalty on The Adinath Co-Operative Bank Limited, Dist. Surat, Gujarat. This information is found in the document named PR1914ADINATH25D3CEFC29704A9289D6D7BDA04708E5.pdf, on page 1. 
 


...

*** User Question *** :  what about april 2023 ?


*** Bot Response *** :  The Monetary Policy Committee (MPC) meeting that occurred in April 2023 decided to keep the policy repo rate unchanged at 6.50 percent. This decision was unanimously voted for by all members of the MPC, which included Dr. Shashanka Bhide, Dr. Ashima Goyal, Prof. Jayanth R. Varma, Dr. Rajiv Ranjan, Dr. Michael Debabrata Patra, and Shri Shaktikanta Das. The committee also decided to remain focused on the withdrawal of accommodation to ensure that inflation progressively aligns with the target while supporting growth. Prof. Jayanth R. Varma expressed reservations on this part of the resolution.

This information can be found in the document named "PR88MINUTESOFMPC507ABE3E36B346B6A843FCDCCD9F6207.pdf" on page 4, sections 14 to 16. 
 


*** User Question *** :   summarize this meeting for me


*** Bot Response *** :  The key decisions and points from the April 2023 Monetary Policy Committee (MPC) meeting are as follows:

1. **Policy Rates Unchanged**: The MPC decided to keep the policy repo rate unchanged at 6.50 percent. Similarly, the standing deposit facility (SDF) rate remains at 6.25 percent, and the marginal standing facility (MSF) rate and the Bank Rate at 6.75 percent.

2. **Focus on Inflation and Growth**: The committee resolved to remain focused on the withdrawal of accommodation to ensure that inflation progressively aligns with the target while supporting growth. This decision aligns with the objective of achieving the medium-term target for consumer price index (CPI) inflation of 4 percent within a band of +/- 2 percent, also while supporting growth.

3. **Unanimous Voting**: All members of the MPC unanimously voted to keep the policy repo rate unchanged at 6.50 percent. However, Prof. Jayanth R. Varma expressed reservations about the part of the resolution focu

*** User Question *** :  Thank you
